# Green Light Meeting after May 29th 
Ideas collected after green light meeting on May 29th. 

In [1]:
using ForwardDiff
using QuadGK
using HCubature 
using StaticArrays 
using LinearAlgebra
using BenchmarkTools
using Plots 

## Section 1: hcubature for 3D integrals result in NaN  
Q: Galerkin using uniform weighting functions (cfr. Morandi-2010) requires three-dimensional integrals dx dy dz. A NaN is encountered when evaluating these integrals using the hcubature function. The NaN error message disappears when integrating a small quantity away from the lower and upper bounds. 

Note that: 
1. Morandi-2010 computes these integrals using a quadrature rule with predefined quadrature points and weights. The hcubature function instead performs an adaptive integral computation. This in part explains the confusion; 
2. The NaN error message is related to the 2D/3D nature of the integral. In merely 1D, hcubature does much better in handling the singularity of the integral;

Dixit Steven G. Johnson (stevengj) in [this Discourse thread](https://discourse.julialang.org/t/how-do-i-choose-between-quadgk-and-cubature-when-i-do-singular-integral-of-a-complex-valued-function/94198) : That being said, if you are doing lots of near-singular integrals like this, especially in higher dimensions, you should consider using a semi-analytical singularity-subtraction procedure to handle the near-singular part analytically and only use quadrature for the rest. This is a common procedure in integral-equation methods, for example, where the integrands often have integrable singularities. 

Below the same error message is reproced on a small example. 

In [2]:
hcubature(x -> log(1/abs((x[1]-.5))^13), (0,), (1,))

(21.988908597000837, 0.12438925065792539)

In [3]:
hcubature(x -> log(1/abs((x[1]-.5))^13), (0,0), (1,2)) 

(43.80483843339491, 0.8177138447179466)

###  Section 1.1: The dimension of the integration matters 

In [5]:
# assuming that destination location is the origin (0,0,0)
hcubature(x -> x[1]/norm(x)^3, (0,0), (1,1))

(Inf, NaN)

In [6]:
# assuming that destination location is the origin (0,0,0)
hcubature(x -> x[1]/norm(x)^3, (0,0,0), (1,1,1)) 

(0.9693880526621552, 1.4444861385160079e-8)

In [7]:
# taking the derivative of the previous case 
hcubature(x -> 1/norm(x)^3, (0,0,0), (1,1,1)) 

(Inf, NaN)

## Section 2: Transform Integrals to Polar Coordinates 
How to handle the phi coordinate? Semi-analytical approach. 